In [1]:
from Model.model import ModelSTGCN
import torch
from torch.utils.data import DataLoader,Dataset
import pandas as pds
import numpy as np
import torch.nn as nn

c:\Users\ADMIN\anaconda3\envs\pythonProject\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class ActionDataset(Dataset):
    def __init__(self,Data,Transform=None):
        super().__init__()
        self.N_frame=Data['total_frames']
        kpscore=np.expand_dims(Data['kp_score'][0],axis=2)
        kp=Data['kp'][0]
        self.data=np.concatenate((kp,kpscore),axis=2)
        self.transform = Transform

    def __getitem__(self, index):
        sample = self.data[index]
        if self.transform:
            sample=self.transform(sample)
        return sample
    def __len__(self):
        return self.N_frame   

In [3]:
class ToTensor():
    def __call__(self,sample):
        return torch.from_numpy(sample)

In [4]:
# kp=datalst[0]['kp'][0]
# kp_score=datalst[0]['kp_score'][0]
# kp_score=np.expand_dims(kp_score,axis=2)
# a=np.concatenate((kp,kp_score),axis=2)
# a


In [20]:
batch_size=4
datalst=pds.read_pickle('train.pkl')
actDataset=ActionDataset(datalst[0],ToTensor())
data=actDataset[5]

torch.Size([715, 17, 3])

In [6]:
batch_size=4
datalst=pds.read_pickle('train.pkl')
actDataset=ActionDataset(datalst[0])
dataloader=DataLoader(dataset=actDataset,batch_size=batch_size,shuffle=True)
n_sample=len(actDataset)
epochs=10
n_iter=round(n_sample/batch_size)
for epoch in range(epochs-8):
    for i,data in enumerate (dataloader):
        if (i+1)% (batch_size)==0:
            print(f'Epoch: {epoch}, Step: {i+1}/{n_iter}, Input shap: {data.shape}')

Epoch: 0, Step: 4/179, Input shap: torch.Size([4, 17, 3])
Epoch: 0, Step: 8/179, Input shap: torch.Size([4, 17, 3])
Epoch: 0, Step: 12/179, Input shap: torch.Size([4, 17, 3])
Epoch: 0, Step: 16/179, Input shap: torch.Size([4, 17, 3])
Epoch: 0, Step: 20/179, Input shap: torch.Size([4, 17, 3])
Epoch: 0, Step: 24/179, Input shap: torch.Size([4, 17, 3])
Epoch: 0, Step: 28/179, Input shap: torch.Size([4, 17, 3])
Epoch: 0, Step: 32/179, Input shap: torch.Size([4, 17, 3])
Epoch: 0, Step: 36/179, Input shap: torch.Size([4, 17, 3])
Epoch: 0, Step: 40/179, Input shap: torch.Size([4, 17, 3])
Epoch: 0, Step: 44/179, Input shap: torch.Size([4, 17, 3])
Epoch: 0, Step: 48/179, Input shap: torch.Size([4, 17, 3])
Epoch: 0, Step: 52/179, Input shap: torch.Size([4, 17, 3])
Epoch: 0, Step: 56/179, Input shap: torch.Size([4, 17, 3])
Epoch: 0, Step: 60/179, Input shap: torch.Size([4, 17, 3])
Epoch: 0, Step: 64/179, Input shap: torch.Size([4, 17, 3])
Epoch: 0, Step: 68/179, Input shap: torch.Size([4, 17, 3])

In [7]:
import math
math.ceil(len(actDataset)/batch_size)

179

In [8]:
batch_size=4
datalst=pds.read_pickle('train.pkl')
actDataset=ActionDataset(datalst[0])
dataloader=DataLoader(dataset=actDataset,batch_size=batch_size,shuffle=True)
n_sample=len(actDataset)
n_iter=round(n_sample/batch_size)
model = ModelSTGCN(3,5)
lr = 1e-3
optim = torch.optim.Adam(model.parameters(),lr)
lr_schedule = torch.optim.lr_scheduler.MultiStepLR(optim,[15,30],gamma=0.1)
epochs = 10
for epoch in range(epochs):
    for i,data in enumerate (dataloader):
        if (i+1)% (batch_size)==0:
            print(f'Epoch: {epoch}, Step: {i+1}/{n_iter}')

Epoch: 0, Step: 4/179
Epoch: 0, Step: 8/179
Epoch: 0, Step: 12/179
Epoch: 0, Step: 16/179
Epoch: 0, Step: 20/179
Epoch: 0, Step: 24/179
Epoch: 0, Step: 28/179
Epoch: 0, Step: 32/179
Epoch: 0, Step: 36/179
Epoch: 0, Step: 40/179
Epoch: 0, Step: 44/179
Epoch: 0, Step: 48/179
Epoch: 0, Step: 52/179
Epoch: 0, Step: 56/179
Epoch: 0, Step: 60/179
Epoch: 0, Step: 64/179
Epoch: 0, Step: 68/179
Epoch: 0, Step: 72/179
Epoch: 0, Step: 76/179
Epoch: 0, Step: 80/179
Epoch: 0, Step: 84/179
Epoch: 0, Step: 88/179
Epoch: 0, Step: 92/179
Epoch: 0, Step: 96/179
Epoch: 0, Step: 100/179
Epoch: 0, Step: 104/179
Epoch: 0, Step: 108/179
Epoch: 0, Step: 112/179
Epoch: 0, Step: 116/179
Epoch: 0, Step: 120/179
Epoch: 0, Step: 124/179
Epoch: 0, Step: 128/179
Epoch: 0, Step: 132/179
Epoch: 0, Step: 136/179
Epoch: 0, Step: 140/179
Epoch: 0, Step: 144/179
Epoch: 0, Step: 148/179
Epoch: 0, Step: 152/179
Epoch: 0, Step: 156/179
Epoch: 0, Step: 160/179
Epoch: 0, Step: 164/179
Epoch: 0, Step: 168/179
Epoch: 0, Step: 17